# Reading for the presentation...

1. https://en.bitcoin.it/wiki/Privacy Read about "Unnecessary input heuristic"

## 1. Small Example

In [ ]:
from heuristics import *
from data_util import *
from cluster import *

input1 = Input("address1", 2)
input2 = Input("address2", 3)
output1 = Output("address5", 4)
output2 = Output("address6", 1)

t1 = Transaction([input1, input2], [output1, output2])

my_clusters = cluster([t1], heuristics.optimal_change)
print(my_clusters)

In [ ]:
visualize_clusters(my_clusters)

## 2. Format raw data
Data is obtained from: https://www.kaggle.com/xblock/bitcoin-partial-transaction-dataset

In [ ]:
# parse_data('archive/dataset3_2016_1_1500000')

## 3. Run The Clustering Algorithms

In [ ]:
import cluster
from data_util import load_data
import heuristics

import pickle
import matplotlib.pyplot as plt
import numpy as np
import time
from collections import defaultdict

In [ ]:
# Load pickled data
data = load_data('dataset3_2016_1_1500000.pkl')

In [ ]:
def cluster_and_pickle(heuristic, threshold=None):
    clusters = cluster.cluster(data, heuristic, threshold)
    
    file_name = 'dataset3_' + heuristic.__name__ + '.pkl'

    if not clusters:
        with open(file_name, 'rb') as f:
            clusters = pickle.load(f)
    else:
        with open(file_name, 'wb') as f:
            pickle.dump(clusters, f)
            
    return clusters
    
def plot_and_analyze(clusters_list, title):
    # Count the number of clusters at a given size
    cluster_sizes = defaultdict(int)
    total_addr = 0
    for cluster in clusters_list:
        cluster_size = len(cluster)
        total_addr += cluster_size
        cluster_sizes[cluster_size] += 1

    sizes = list(cluster_sizes.keys())
    counts = list(cluster_sizes.values())
    avg_size = sum(size * count for size, count in zip(sizes, counts)) / sum(counts)
    avg_non_single_size = sum(size * count for size, count in zip(sizes, counts) if size > 1) / \
                          sum(count for size, count in zip(sizes, counts) if size > 1)
    print('Total number of addresses:          ', total_addr)
    print('Total number of clusters:           ', len(clusters_list))
    print('Number of single-address clusters:  ', cluster_sizes[1])
    print('Size of largest cluster:            ', max(sizes))
    print('Average cluster size:               ', avg_size)
    print('Average cluster size (excluding single-address clusters): ', avg_non_single_size)
    
    # Get cluster size counts in particular range
    x, y = zip(*[(size, count) for size, count in cluster_sizes.items() if 100 > size > 1])

    # Plot
    plt.title(title)
    plt.figure(figsize=(20,10))
    plt.bar(x, y)
    plt.ylabel('Number of Clusters')
    plt.xlabel('Number of addresses in cluster')
    plt.yscale('log')

In [ ]:
multi_input_clusters = cluster_and_pickle(heuristics.multi_input)

In [ ]:
optimal_change_clusters = cluster_and_pickle(heuristics.optimal_change)

In [ ]:
multi_input_opimal_change_clusters = cluster_and_pickle(heuristics.multi_input_optimal_change)

In [ ]:
shadow_clusters = cluster_and_pickle(heuristics.shadow)

### Hopefully don't need this section

# Obtain list of clusters from data
start_time = time.time()
clusters = cluster.cluster(data, multi_input)
print('Elapsed Time:', time.time() - start_time)

# Save/Load cluster data for convenience
file_name = 'dataset3_muti_input.pkl'

if not clusters:
    with open(file_name, 'rb') as f:
        clusters = pickle.load(f)
else:
    with open(file_name, 'wb') as f:
        pickle.dump(clusters, f)

# Count the number of clusters at a given size
cluster_sizes = defaultdict(int)
total_addr = 0
for cluster in clusters:
    cluster_size = len(cluster)
    total_addr += cluster_size
    cluster_sizes[cluster_size] += 1

sizes = list(cluster_sizes.keys())
counts = list(cluster_sizes.values())
avg_size = sum(size * count for size, count in zip(sizes, counts)) / sum(counts)
avg_non_single_size = sum(size * count for size, count in zip(sizes, counts) if size > 1) / \
                      sum(count for size, count in zip(sizes, counts) if size > 1)
print('Total number of addresses:          ', total_addr)
print('Total number of clusters:           ', len(clusters))
print('Number of single-address clusters:  ', cluster_sizes[1])
print('Size of largest cluster:            ', max(sizes))
print('Average cluster size:               ', avg_size)
print('Average cluster size (excluding single-address clusters): ', avg_non_single_size)

# Get cluster size counts in particular range
x, y = zip(*[(size, count) for size, count in cluster_sizes.items() if 100 > size > 1])

# Plot
plt.figure(figsize=(20,10))
plt.bar(x, y)
plt.ylabel('Number of Clusters')
plt.xlabel('Number of addresses in cluster')
plt.yscale('log')

## 4. Make a histogram for shadow

In [ ]:
# this is ticket number 13 in github

# run __get_shadow_data(transactions) on the real data
# then make a histogram of the output

# based on the histogram, do you think we have a clear distinction of who is a merchant? 
# Or will we need to make an assumption on our own?

In [ ]:
start_time = time.time()
data = parse_data('archive/dataset3_2016_1_1500000')
print('Elapsed Time:', time.time() - start_time)

In [ ]:
# Load pickled data
start_time = time.time()
data = load_data('dataset3_2016_1_1500000.pkl')
print('Elapsed Time:', time.time() - start_time)

In [ ]:
results = heuristics.__get_shadow_data(data)

In [ ]:
results

In [ ]:
addresses = []
merchant_counts = []
for r in results:
    addresses.append(r)
    merchant_counts.append(results[r])

In [ ]:
counts = {}

for c in merchant_counts:
    if c in counts:
        counts[c] += 1
    else:
        counts[c] = 1

print(counts)

In [ ]:
plt.plot(list(counts.values())[3:100])

## Compare Clusters

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score # maybe use adjusted_rand_score if need
# [[1, 2, 3], [4, 5], [6]] -> [0, 0, 0, 1, 1, 2]
adjusted_rand_score([0, 0, 2, 2], [1, 1, 0, 0])

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
# implement split/join